In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import *
import matplotlib.pyplot as plt
import pyrosm
from pyrosm.data import sources
from shapely import wkt
import seaborn as sns
import glob

/home/jsepulveda/miniconda3/envs/myenv/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
usuarios_objetivo = pd.read_csv('/home/jsepulveda/usuarios_objetivo_finales.csv')

In [3]:
usuarios_objetivo_lista=tuple(usuarios_objetivo['user'].to_list())

In [4]:
# obtained from https://docs.faculty.ai/how_to/spark/local_spark.html
import pyspark

number_cores = 16
memory_gb = 240
tmp_dir = '/home/jsepulveda/' #cambialo a tu directorio home

conf = (
    pyspark.SparkConf()
        .setMaster('local[{}]'.format(number_cores))
        .set('spark.driver.memory', '{}g'.format(memory_gb))
        .set('spark.local.dir', tmp_dir)
        .set("spark.sql.join.preferSortMergeJoin", "true") #force sort join
        .set("spark.sql.autoBroadcastJoinThreshold", "1") #force sort join
)
sc = pyspark.SparkContext(conf=conf)
sql = pyspark.sql.SQLContext(sc)

In [5]:
from glob import glob

In [6]:
# esto es para chequear si el directorio existe
DATA_PATH = '/home/jsepulveda/pre_inauguracion/day=16/'
files = glob(DATA_PATH)
files

['/home/jsepulveda/pre_inauguracion/day=16/']

In [7]:
xdr = sql.read.csv(files, header=True,)
xdr.createOrReplaceTempView("xdr")
xdr.show(5)

+------+------+---+--------------------+--------------------+-------+-------+-------+----+
| fecha|  hora|dur|                numa|                numb|    rin|rinhead|rintail|tipo|
+------+------+---+--------------------+--------------------+-------+-------+-------+----+
|171016|133636|  2|fcb4b64966571d03a...|201d1ad4fde8bea37...|SFPOFU7|  SFPOF|     U7|   D|
|171016|134920|  4|fcb4b64966571d03a...|201d1ad4fde8bea37...|SFPOFU7|  SFPOF|     U7|   D|
|171016|141640|  1|fcb4b64966571d03a...|201d1ad4fde8bea37...|SFPOFU7|  SFPOF|     U7|   D|
|171016|144640|  1|fcb4b64966571d03a...|201d1ad4fde8bea37...|SFPOFU7|  SFPOF|     U7|   D|
|171016|145302| 12|fcb4b64966571d03a...|201d1ad4fde8bea37...|SFPOFU7|  SFPOF|     U7|   D|
+------+------+---+--------------------+--------------------+-------+-------+-------+----+
only showing top 5 rows



In [8]:
test_query = f'select * from xdr where numa in {usuarios_objetivo_lista} and hora between 73000 and 90000'
sql.sql(test_query).show(10)

+------+------+------+--------------------+--------------------+-------+-------+-------+----+
| fecha|  hora|   dur|                numa|                numb|    rin|rinhead|rintail|tipo|
+------+------+------+--------------------+--------------------+-------+-------+-------+----+
|171016|073311|     0|f60785dc54bee7eec...|201d1ad4fde8bea37...|ATICFL1|  ATICF|     L1|   D|
|171016|075312|     0|f60785dc54bee7eec...|201d1ad4fde8bea37...|ATICFL1|  ATICF|     L1|   D|
|171016|081314|     0|f60785dc54bee7eec...|201d1ad4fde8bea37...|ATICFL1|  ATICF|     L1|   D|
|171016|083316|     0|f60785dc54bee7eec...|201d1ad4fde8bea37...|ATICFL1|  ATICF|     L1|   D|
|171016|085317|     0|f60785dc54bee7eec...|201d1ad4fde8bea37...|ATICFL1|  ATICF|     L1|   D|
|171016|080459| 11912|816a50e5839bd619c...|201d1ad4fde8bea37...|VDNOFL1|  VDNOF|     L1|   D|
|171016|082645| 10416|816a50e5839bd619c...|201d1ad4fde8bea37...|MEBL5L1|  MEBL5|     L1|   D|
|171016|084205|102416|816a50e5839bd619c...|201d1ad4fde8bea37

In [10]:
pre_punta_mañana = sql.sql(test_query)
pre_punta_mañana.write.parquet('pre_punta_mañana_dia_1.parquet', compression='snappy')
